In [ ]:
!curl -s -X DELETE http://localhost:9200/ubi_queries/
!curl -s -X DELETE http://localhost:9200/ubi_events/

In [ ]:
from time import time
import json
import random
import uuid

from opensearchpy import OpenSearch

client = OpenSearch("http://localhost:9200", use_ssl=False)

## Parameterize the generation of UBI queries and events

In [ ]:
search_terms = ['computer', 'laptop', 'notebook', 'desk', 'power plug']
actions = ["click_through", "add_to_cart", "click", "watch", "view", "purchase"]

object_id_field = "ean"
item_description_field = "title"
index_name = "ecommerce"
number_of_user_searches = 1000
page_size = 20
max_number_events_for_each_search = 10

## Generate the UBI queries and events

In [ ]:
for x in range(number_of_user_searches):

    random_search_term_index = random.randint(0, len(search_terms) - 1)
    random_search_term = search_terms[random_search_term_index]
    
    client_id = str(uuid.uuid4())
    query_id = str(uuid.uuid4())
    
    query = {
       "from": 0,
        "size": page_size,
       "query": {
         "match_all": {}
       },
        "ext": {
            "ubi": {
                "client_id": client_id,
                "query_id": query_id,
                "user_query": random_search_term
            }
        }
     }
    
    query_response = client.search(
        body = query,
        index = index_name
    )

    random_number_of_events = random.randint(0, max_number_events_for_each_search - 1)

    for y in range(random_number_of_events):
            
        random_search_result_index = random.randint(0, page_size - 1)
        random_action_index = random.randint(0, len(actions) - 1)
        session_id = str(uuid.uuid4())
        
        #print("random_action_index = " + str(random_action_index))
        #print("random_search_result_index = " + str(random_search_result_index))

        ubi_event = {
            "action_name": actions[random_action_index],
            "client_id": client_id,
            "query_id": query_id,
            "message_type": None,
            "message": None,
            "timestamp": time(),
            "event_attributes": {
                "object": {
                    "object_id_field": "ean",
                    "object_id": query_response["hits"]["hits"][random_search_result_index]["_source"][object_id_field],
                    "description": query_response["hits"]["hits"][random_search_result_index]["_source"][item_description_field]
                },
                "position": {
                    "index": random_search_result_index
                },
                "session_id": session_id
            }
        }
    
        event_id = str(uuid.uuid4())
        
        response = client.index(
            body = ubi_event,
            index = "ubi_events",
            id = event_id,
            refresh = True
        )

In [ ]:
!curl -s http://localhost:9200/ubi_queries/_count | jq
!curl -s http://localhost:9200/ubi_events/_count | jq

In [ ]:
!curl -s http://localhost:9200/ubi_queries/_search | jq

In [ ]:
!curl -s http://localhost:9200/ubi_events/_search | jq